In [ ]:
import os
import sys
import urllib, io
sys.path.append("..")

from os import listdir
from os.path import isfile, join

import numpy as np
import scipy.stats as stats
import pandas as pd
from random import random

import pymongo as pm
from collections import Counter
import json
import re
import ast

from PIL import Image, ImageOps, ImageDraw, ImageFont 

from io import BytesIO
import base64

import  matplotlib
from matplotlib import pylab, mlab, pyplot
%matplotlib inline
from IPython.core.pylabtools import figsize, getfigs
plt = pyplot
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42

import seaborn as sns
sns.set_context('talk')
sns.set_style('darkgrid')

from IPython.display import clear_output

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")

import blockworld_helpers as utils


In [ ]:
## directory & file hierarchy
proj_dir = os.path.abspath('../..')
datavol_dir = os.path.join(proj_dir,'data')
analysis_dir = os.path.abspath(os.path.join(os.getcwd(),'..'))
results_dir = os.path.join(analysis_dir,'results')
stim_dir = os.path.join(proj_dir,'stimuli/interesting_structures')
plot_dir = os.path.join(results_dir,'plots')
csv_dir = os.path.join(results_dir,'csv')
json_dir = os.path.join(results_dir,'json')
exp_dir = os.path.abspath(os.path.join(proj_dir,'experiments'))
png_dir = os.path.abspath(os.path.join(datavol_dir,'png'))
jefan_dir = os.path.join(analysis_dir,'jefan')
will_dir = os.path.join(analysis_dir,'will')

## add helpers to python path
if os.path.join(proj_dir,'stimuli') not in sys.path:
    sys.path.append(os.path.join(proj_dir,'stimuli'))
    
if not os.path.exists(results_dir):
    os.makedirs(results_dir)
    
if not os.path.exists(plot_dir):
    os.makedirs(plot_dir)   
    
if not os.path.exists(csv_dir):
    os.makedirs(csv_dir)       

In [ ]:
os.getcwd()

In [ ]:
iterationName = 'pilot4'

# Trial_end data
trial_path = os.path.join(csv_dir,'block_silhouette_{}.csv'.format(iterationName))
df = pd.read_csv(trial_path)

# Settled_block data
settled_path = os.path.join(csv_dir,'block_silhouette_settled_{}.csv'.format(iterationName))
dfs = pd.read_csv(settled_path)

# Explore_end data
explore_path = os.path.join(csv_dir,'block_silhouette_explore_{}.csv'.format(iterationName))
dfe = pd.read_csv(explore_path)

# Exclude outliers
low_scores = df[(df.trialNum == 15) & (df.score == 0)]['gameID']

df = df[~df.gameID.isin(low_scores)]
dfs = dfs[~dfs.gameID.isin(low_scores)]
dfe = dfe[~dfe.gameID.isin(low_scores)]

n = df.gameID.nunique()
print(str(n) + ' participants with score > 0 and full set of trials')

In [ ]:
######################### SOME DRAWING FUNCTIONS ##########################
###########################################################################    

def get_patch(verts,
              color='orange',
              line_width = 2):
    '''
    input:
        verts: array or list of (x,y) vertices of convex polygon. 
                last vertex = first vertex, so len(verts) is num_vertices + 1
        color: facecolor
        line_width: edge width    
    output:
        patch matplotlib.path patch object
    '''
    codes = [1] + [2]*(len(verts)-1)    ## 1 = MOVETO, 2 = LINETO
    path = Path(verts,codes)
    patch = patches.PathPatch(path, facecolor=color, lw=line_width)
    return patch

def get_patches_solution(blocks):
    patches = []
    for (b) in blocks:
        patches.append(get_patch(b,color='#29335C'))
    return patches

def get_patches_stim(blocks):
    patches = []
    for (b) in blocks:
        patches.append(get_patch(b.verts,color=b.base_block.color))
    return patches


def render_blockworld(patches,
                      xlim=(0,world_size),
                      ylim=(0,world_size),
                      figsize=(4,4)):   
    '''
    input: 
        patches: list of patches generated by get_patch() function
        xlim, ylim: axis limits
        figsize: defaults to square aspect ratio
    output:
        visualization of block placement
    '''
    fig = plt.figure(figsize=figsize)
    ax = fig.add_subplot(111)
    for patch in patches:
        ax.add_patch(patch)
    ax.set_xlim(xlim)
    ax.set_ylim(ylim) 
    cur_axes = plt.gca()
    cur_axes.axes.get_xaxis().set_visible(False)
    cur_axes.axes.get_yaxis().set_visible(False)        
    #plt.show()
    return fig

def draw_solution(vertices, world_size=900):
    fig = render_blockworld(get_patches_solution(vertices),
                            xlim=(0,world_size),
                            ylim=(0,world_size)) 
    return fig

def draw_stim(world, world_size=12):
    fig = render_blockworld(get_patches_stim(world.blocks),
                            xlim=(-2,10),
                            ylim=(-2,10)) 
    return fig

def render_blockworld_subplot(patches,
                              ax,
                              xlim=(0,world_size),
                              ylim=(0,world_size),
                              figsize=(4,4)
                            ):   
    '''
    input: 
        patches: list of patches generated by get_patch() function
        xlim, ylim: axis limits
        figsize: defaults to square aspect ratio
    output:
        visualization of block placement
    '''
    for patch in patches:
        ax.add_patch(patch)
    ax.set_xlim(xlim)
    ax.set_ylim(ylim) 
    cur_axes = plt.gca()
    cur_axes.axes.get_xaxis().set_visible(False)
    cur_axes.axes.get_yaxis().set_visible(False)
    return ax

# Draw final structure for a participant
def draw_final_structure(df, gameID, target_name):
    vert_dict = df.loc[(df.gameID == gameID) & (df.targetName == target_name),'allVertices'].apply(ast.literal_eval).values[0]
    vertices = compress_vertices(vert_dict)
    score = df.loc[(df.gameID == gameID) & (df.targetName == target_name),'normedScore'].values[0]
    return draw_solution(vertices)
    
def draw_final_structure_subplot(df, gameID, target_name, ax):
    vert_dict = df.loc[(df.gameID == gameID) & (df.targetName == target_name),'allVertices'].apply(ast.literal_eval).values[0]
    vertices = compress_vertices(vert_dict)
    score = df.loc[(df.gameID == gameID) & (df.targetName == target_name),'normedScore'].values[0]
    return render_blockworld_subplot(get_patches_solution(vertices),
                                        ax,
                                        xlim=(0,world_size),
                                        ylim=(0,world_size)) 


In [ ]:
def get_light_df(df_full):
    ''' Get most essential columns from dataframe
    '''
    df_light = df_full[['randID','trialNum','phase','condition','eventType','score','normedScore','numBlocks']]
    return df_light

def compress_vertices(vert_dict):
    '''
    '''
    vert_list = list(map(lambda block: list(map(lambda corner: (corner['x'],world_size-corner['y']), block)), vert_dict))
    return vert_list
    
def get_world_vertices(row):
    ''' Get vertices of this row.
    '''
    vert_dict = row['allVertices'] # extract dictionary of vertices of blocks 
    world_verts = compress_vertices(vert_dict) # convert dictionary to list of unlabelled tuples
    return world_verts

def draw_world_from_row(row):
    ''' Renders state of world from one observation
    '''
    world_verts = get_final_vertices(row)
    draw_world(world_verts)

#def draw_block_in_context(row):
    ''' draw new block in figure showing old blocks in a different color
    '''


In [ ]:
# Print all final structure for one participant
sub1 = df.head(1).gameID
dfSubjectTrials = df[df.gameID.isin(sub1)]
for i in range(0,len(dfSubjectTrials)-1):
    vert_dict = dfSubjectTrials.loc[dfSubjectTrials.trialNum == i,'allVertices'].apply(ast.literal_eval).values[0]
    vertices = compress_vertices(vert_dict)
    draw_world(vertices)

In [ ]:
df.targetName.unique()

In [ ]:
# draw all final structure for a given target
ppts = df.gameID.unique()
ppt = 0
gameID = ppts[ppt]
target_name = 'hand_selected_001'

for i in range(0,len(df.gameID.unique())-1):
    gameID = ppts[i]
    drawFinalStructure(df,gameID,target_name)

In [ ]:
# draw all final structure for all participants
ppts = df.gameID.unique()
ppts.sort()

target_names = [f.split('.')[0] for f in listdir(stim_dir) if isfile(join(stim_dir, f))]    
target_names.sort()

fig = plt.figure(figsize=(40, 80))
columns = 16
rows = 32

k = 0
for i in range(0,len(ppts)):
    gameID = ppts[i]
    for j in range(0,len(image_names)):
        target_name = target_names[j]
        k += 1
        ax = fig.add_subplot(rows, columns, k)
        ax = draw_final_structure_subplot(df, gameID, target_name, ax)
plt.show()

In [ ]:
# Display in grid
# Attach labels

In [ ]:
# Retrieve stimuli
image_file_names = [f for f in listdir(stim_dir) if isfile(join(stim_dir, f))]    
image_file_names.sort()

fig=plt.figure(figsize=(20, 20))
columns = 4
rows = 4
for i in range(0, columns*rows):
    J = open(join(stim_dir,image_file_names[i])).read()
    #fig.add_subplot(rows, columns, i)
    w = utils.World()
    w.populate_from_json(J)
    subfig = draw_stim(w)
    #fig.add_subplot(rows, columns, i)
    #plt.imshow(img)
plt.show()